In [9]:
!pip install langchain langchain-community pinecone pinecone-text pinecone-notebooks langchain_huggingface

In [2]:
api_key = "" # add your pinecone API key here

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

## initialize pinecone client
pc = Pinecone(api_key = api_key)

# create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384, # dimension of dense vector
        metric = "dotproduct", # sparse values supported only for dotproduct
        spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
    )

In [6]:
index = pc.Index(index_name)
index

In [11]:
# vector embedding and sparse matrix
import os
import getpass

if not os.environ.get('HUGGINGFACEHUB_API_TOKEN'):
    os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass('Hugging Face Token:')

Hugging Face Token:··········


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
sentences = [
    "In 2023 I visited Paris",
    "In 2022 I visited New York",
    "In 2021 I visited New Orleans",
]

# applying tfidif values on sentences
bm25_encoder.fit(sentences)

# store values in a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
retriever = PineconeHybridSearchRetriever(
    embeddings = embeddings,
    sparse_encoder = bm25_encoder,
    index = index
)

In [16]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7a3907dfb8c0>, index=<pinecone.db_data.index.Index object at 0x7a3ab4d196d0>)

In [17]:
retriever.add_texts(
    [
        "In 2023 I visited Paris",
        "In 2022 I visited New York",
        "In 2021 I visited New Orleans",
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
retriever.invoke("What city did I visit last")

[Document(metadata={'score': 0.280160069}, page_content='In 2021 I visited New Orleans'),
 Document(metadata={'score': 0.258902192}, page_content='In 2022 I visited New York'),
 Document(metadata={'score': 0.231151313}, page_content='In 2023 I visited Paris')]